# Kernel Latent Variable Time-varying Graphical Lasso
More than 1-Markovian!

In [14]:
%matplotlib inline
from __future__ import print_function, division

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from itertools import product
from functools import partial
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.datasets.base import Bunch
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import GraphLasso, empirical_covariance
from sklearn.datasets.base import Bunch
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.gaussian_process import kernels

from regain import prox; reload(prox)
from regain.covariance import time_graph_lasso_; reload(time_graph_lasso_);
from regain.covariance import latent_time_graph_lasso_; reload(latent_time_graph_lasso_);
from regain import utils; reload(utils)
import time

In [15]:
from regainpr.bayesian import wishart_process_; reload(wishart_process_)
from regainpr.covariance import kernel_latent_time_graph_lasso_; reload(kernel_latent_time_graph_lasso_);
from regainpr.covariance import kernel_time_graph_lasso_; reload(kernel_time_graph_lasso_);

from regainpr import datasets; reload(datasets);

In [16]:
def tgl_results(data_grid, K, **params):
    mdl = time_graph_lasso_.TimeGraphLasso(
        time_on_axis='last', assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
        max_iter=500, rho=1./ np.sqrt(data_grid.shape[0]))

    tic = time.time()
    ll = mdl.set_params(**params).fit(data_grid)
    tac = time.time()
    iterations = ll.n_iter_
    MSE_precision = utils.error_norm(K, ll.precision_)
    ss = utils.structure_error(K, ll.precision_)#, thresholding=1, eps=1e-5)

    res = dict(n_dim_obs=K.shape[1],
               time=tac-tic,
               iterations=iterations,
               MSE_precision=MSE_precision,
               likelihood=mdl.score(data_grid),
               estimator=ll)
    res = dict(res, **ss)
    return res

def ktgl_results(data_grid, K, **params):
    mdl = kernel_time_graph_lasso_.KernelTimeGraphLasso(
        time_on_axis='last', assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
        max_iter=500, rho=1./ np.sqrt(data_grid.shape[0]), update_rho_options=dict(mu=5))

    tic = time.time()
    ll = mdl.set_params(**params).fit(data_grid)
    tac = time.time()
    iterations = ll.n_iter_
    MSE_precision = utils.error_norm(K, ll.precision_)
    ss = utils.structure_error(K, ll.precision_)#, thresholding=1, eps=1e-5)

    res = dict(n_dim_obs=K.shape[1],
               time=tac-tic,
               iterations=iterations,
               MSE_precision=MSE_precision,
               likelihood=mdl.score(data_grid),
               estimator=ll)
    res = dict(res, **ss)
    return res

def new_ktgl_results(data_list, K, **params):
    mdl = kernel_time_graph_lasso_.NewKernelTimeGraphLasso(
        alpha=0.5, psi='laplacian',
        assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
        max_iter=500, rho=1./ np.sqrt(data_grid.shape[0]), update_rho_options=dict(mu=5),
        kernel=partial(kernels.ExpSineSquared, periodicity=np.pi), length_scale=2)

    X = np.vstack(data_list)
    y = np.array([
        np.ones(x.shape[0]) *i for i, x in enumerate(data_list)]).flatten().astype(int)
    
    tic = time.time()
    ll = mdl.set_params(**params).fit(X, y)
    tac = time.time()
    iterations = ll.n_iter_
    MSE_precision = utils.error_norm(K, ll.precision_)
    ss = utils.structure_error(K, ll.precision_)#, thresholding=1, eps=1e-5)

    res = dict(n_dim_obs=K.shape[1],
               time=tac-tic,
               iterations=iterations,
               MSE_precision=MSE_precision,
               likelihood=mdl.score(X, y),
               estimator=ll)
    res = dict(res, **ss)
    return res

def kltgl_results(data_grid, K, K_obs, ells, **params):
    mdl = kernel_latent_time_graph_lasso_.KernelLatentTimeGraphLasso(
        time_on_axis='last', assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
        max_iter=1000, rho=1./ np.sqrt(data_grid.shape[0]),
        update_rho_options=dict(mu=5))

    tic = time.time()
    ll = mdl.set_params(**params).fit(data_grid)
    tac = time.time()
    iterations = ll.n_iter_
    ss = utils.structure_error(K, ll.precision_)
    MSE_observed = utils.error_norm(K_obs, ll.precision_ - ll.latent_)
    MSE_precision = utils.error_norm(K, ll.precision_, upper_triangular=True)
    MSE_latent = utils.error_norm(ells, ll.latent_)
    mean_rank_error = utils.error_rank(ells, ll.latent_)

    res = dict(n_dim_obs=K.shape[1],
               time=tac-tic,
               iterations=iterations,
               MSE_precision=MSE_precision,
               MSE_observed=MSE_observed,
               MSE_latent=MSE_latent,
               mean_rank_error=mean_rank_error,
               note=None,
               estimator=ll,
               likelihood=mdl.score(data_grid),
              latent=ll.latent_)

    res = dict(res, **ss)
    return res

def wp_results(data_grid, K, K_obs, ells, **params):
    n_iter = 1000
    mdl = wishart_process_.WishartProcess(n_iter=n_iter)

    tic = time.time()
    ll = mdl.fit(data_grid)
    tac = time.time()
    iterations = n_iter
    ss = utils.structure_error(K, ll.precision_)
    
    MSE_precision = utils.error_norm(K, ll.precision_, upper_triangular=True)

    res = dict(n_dim_obs=K.shape[1],
               time=tac-tic,
               iterations=iterations,
               MSE_precision=MSE_precision,
               estimator=ll,
               likelihood=mdl.score(data_grid),
              latent=ll.latent_)

    res = dict(res, **ss)
    return res

def lwp_results(data_grid, K, K_obs, ells, **params):
    n_iter = 1000
    mdl = wishart_process_.WishartProcess(n_iter=n_iter)

    tic = time.time()
    ll = mdl.fit(data_grid)
    tac = time.time()
    iterations = n_iter
    ss = utils.structure_error(K, ll.precision_)
    
    MSE_observed = utils.error_norm(K_obs, ll.precision_ - ll.latent_)
    MSE_precision = utils.error_norm(K, ll.precision_, upper_triangular=True)
    MSE_latent = utils.error_norm(ells, ll.latent_)
    mean_rank_error = utils.error_rank(ells, ll.latent_)

    res = dict(n_dim_obs=K.shape[1],
               time=tac-tic,
               iterations=iterations,
               MSE_precision=MSE_precision,
               MSE_observed=MSE_observed,
               MSE_latent=MSE_latent,
               mean_rank_error=mean_rank_error,
               note=None,
               estimator=ll,
               likelihood=mdl.score(data_grid),
              latent=ll.latent_)

    res = dict(res, **ss)
    return res

In [17]:
# setting 1
n_samples = 100
T = 10
n_dim_obs = 5

k = (n_dim_obs, T)

np.random.seed(0)

reload(datasets)
data = {(dim, T) : datasets.make_dataset(
    mode='sin', shape='smooth',
    update_theta='l2', normalize_starting_matrices=False,
    n_samples=n_samples, n_dim_lat=0, n_dim_obs=dim,  T=T, epsilon=1e-1,
    proportional=True, degree=2, keep_sparsity=True)
    for dim in [n_dim_obs]}

In [43]:
# setting 2 - sample from GPs
n_samples = 100
T = 10
n_dim_obs = 10

k = (n_dim_obs, T)

np.random.seed(0)   

reload(datasets)
data = {(dim, T) : datasets.make_dataset(
    mode='gp', n_samples=n_samples, n_dim_lat=1, n_dim_obs=dim,  T=T, epsilon=0.4)
    for dim in [n_dim_obs]}

In [44]:
# info on the data set
K = data[k].thetas
print ("Percentual of non-zero components at each time: {}".format(
    [(i!=0).sum() / i.size for i in K]))

print ("Percentual of total non-zero components: {}".format(
    (K != 0).sum() / (n_dim_obs ** 2 * T)))

Percentual of non-zero components at each time: [0.72, 0.68, 0.7, 0.66, 0.7, 0.66, 0.66, 0.72, 0.68, 0.74]
Percentual of total non-zero components: 0.692


In [46]:
# prepare dataframe for results
n_dims = [n_dim_obs]
n_times = [T]
methods = ['TGL', 'KTGL', 'NKTGL', 'KLTGL']
scores = sorted([
    "MSE_precision", "MSE_observed", "MSE_latent", 'estimator', "mean_rank_error",
    'time', 'iterations', 'precision', 'recall', 'accuracy', 'balanced_accuracy',
    'f1', 'npv', 'prevalence', 'miss_rate', 'likelihood',
    'specificity', 'plr', 'nlr'])

cols = pd.MultiIndex.from_product([scores, n_dims], names=('score', 'dim'))
rows = pd.MultiIndex.from_product([methods, n_times], names=('method', 'time'))

dff = pd.DataFrame(columns=cols, index=rows)
idx = pd.IndexSlice

In [47]:
beta = 0.5
rng = np.full(T-1, 1000)
kernel_phi = np.diag(rng,1) +np.diag(rng,-1) + np.eye(T)
kernel_phi = np.full((T ,T), 1000)

In [48]:
from sklearn.gaussian_process import kernels
rbf = kernels.RBF(length_scale=0.5)
expsin = kernels.ExpSineSquared(periodicity=np.pi, length_scale=2)
kernel = expsin(np.arange(T)[:,None])

In [49]:
# setting 1
alpha = 0.6
# beta = kernel[0,1]

for i, (k, res) in enumerate(sorted(data.items())[:5]):
    dim = k[0]
    print("Start with: dim=%d, T=%d (it %d)" % (k[0],k[1], i))
    data_list = res.data
    K = res.thetas
    K_obs = res.thetas_observed
    ells = res.ells
    data_grid = np.array(data_list).transpose(1,2,0)  # to use it later for grid search

    print("starting TGL ...\r", end='')
    res = tgl_results(data_grid, K, beta=beta, alpha=alpha)
    dff.loc[idx['TGL', k[1]], idx[:, k[0]]] = [res.get(x,None) for x in scores]
    
    print("starting KTGL ...\r", end='')
    res = ktgl_results(data_grid, K, kernel=kernel, alpha=alpha)
    dff.loc[idx['KTGL', k[1]], idx[:, k[0]]] = [res.get(x,None) for x in scores]
    
    print("starting NKTGL ...\r", end='')
    res = new_ktgl_results(data_list, K, alpha=alpha, length_scale=0.4)
    dff.loc[idx['NKTGL', k[1]], idx[:, k[0]]] = [res.get(x,None) for x in scores]
    
    print("starting KLTGL ...\r", end='')
    res = kltgl_results(data_grid, K, K_obs, ells, kernel_psi=kernel,
                        kernel_phi=kernel_phi, alpha=.1)
    dff.loc[idx['KLTGL', k[1]], idx[:, k[0]]] = [res.get(x,None) for x in scores]

Start with: dim=10, T=10 (it 0)


In [50]:
mm = dff.xs(n_dim_obs, level='dim', axis=1).xs(T, level='time')
# mm['likelihood']
mm

score,MSE_latent,MSE_observed,MSE_precision,accuracy,balanced_accuracy,estimator,f1,iterations,likelihood,mean_rank_error,miss_rate,nlr,npv,plr,precision,prevalence,recall,specificity,time
method,,,,,,,,,,,,,,,,,,,
TGL,None,None,0.441912,0.728,0.598078,"TimeGraphLasso(alpha=0.6, assume_centered=0, b...",0.826531,68,-5513.8,None,0.0635838,0.244798,0.645161,1.26498,0.739726,0.692,0.936416,0.25974,0.159566
KTGL,None,None,0.61915,0.704,0.539299,"KernelTimeGraphLasso(alpha=0.6, assume_centere...",0.819071,75,-5734.92,None,0.0317919,0.287997,0.607143,1.08835,0.709746,0.692,0.968208,0.11039,0.249415
NKTGL,None,None,0.268091,0.736,0.593049,"NewKernelTimeGraphLasso(alpha=0.6, assume_cent...",0.835,71,-5472.83,None,0.0346821,0.157089,0.73913,1.23882,0.735683,0.692,0.965318,0.220779,0.249345
KLTGL,0.00808687,0.444134,0.290368,0.698,0.511542,"KernelLatentTimeGraphLasso(alpha=0.1, assume_c...",0.820452,172,-5698.25,1.7,0.00289017,0.111272,0.8,1.0237,0.69697,0.692,0.99711,0.025974,1.13335


In [30]:
mdl = kernel_time_graph_lasso_.KernelTimeGraphLasso(
    time_on_axis='last', assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
    max_iter=500, rho=1./ np.sqrt(data_grid.shape[0]), update_rho_options=dict(mu=5))
mdl.fit(data_grid)
mdl.score(data_grid)

-6648.427421079336

In [31]:
X = np.vstack(data_list)
y = np.array([np.ones(x.shape[0]) *i for i, x in enumerate(data_list)]).flatten().astype(int)

In [37]:
reload(prox)
reload(time_graph_lasso_)
reload(kernel_time_graph_lasso_);

from functools import partial

from sklearn.model_selection import StratifiedKFold

from skopt.optimizer import optimizer; reload(optimizer)
from skopt import searchcv; reload(searchcv)
from skopt.space import Categorical
# include below until https://github.com/scikit-optimize/scikit-optimize/issues/718 is resolved
class BayesSearchCV(searchcv.BayesSearchCV):
    def _run_search(self, x): raise BaseException('Use newer skopt')

mdl = kernel_time_graph_lasso_.NewKernelTimeGraphLasso(
    alpha=0.5, psi='laplacian',
    assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
    max_iter=500, rho=1./ np.sqrt(data_grid.shape[0]), update_rho_options=dict(mu=5),
    kernel=partial(kernels.ExpSineSquared, periodicity=np.pi), length_scale=2)

bscv = BayesSearchCV(mdl, search_spaces={
        'alpha': (1e-4, 1e+1, 'log-uniform'),  
        'length_scale': (1e-4, 1e+1, 'log-uniform'),
#         'kernel': Categorical([partial(kernels.ExpSineSquared, periodicity=np.pi),
#                                kernels.RBF]),  # categorical parameter
    },
    n_iter=50, n_points=3, cv=StratifiedKFold(3))

bscv.fit(X, y)
bscv.score(X, y)

-6664.273982392405

In [38]:
bscv.best_params_

{'alpha': 0.5792194050871016, 'length_scale': 0.38781388320536275}

In [ ]:
reload(wishart_process_)
wp = wishart_process_.WishartProcess(time_on_axis='first', verbose=True).fit(data_list)

wp.likelihood(wp.D_map)

wp.loglikes_after_burnin.max()

wp.store_precision = True

wp.score(data_list)

In [ ]:
mm = dff.xs(n_dim_obs, level='dim', axis=1).xs(T, level='time')
mm

In [ ]:
np.abs(dff.estimator[100]['TGL'][10].precision_ - dff.estimator[100]['KTGL'][10].precision_)

In [ ]:
from decimal import Decimal
' & '.join(['%.3f' % Decimal(i) for i in mm['MSE_precision']])

In [ ]:
dff[[s for s in scores if s != 'estimator']].to_pickle("dff_setting_1.pkl")

In [ ]:
l1 = ([np.linalg.matrix_rank(r) for r in mm.estimator['LTGL ($\ell_2^2$)'].latent_])
l2 = ([np.linalg.matrix_rank(r) for r in mm.estimator['LTGL ($\ell_1$)'].latent_])
l3 = ([np.linalg.matrix_rank(r) for r in mm.estimator['LVGLASSO'].L])

l4 = ([np.linalg.matrix_rank(r) for r in mm.estimator['LTGL ($\ell_2^2$)'].latent_])
l5 = ([np.linalg.matrix_rank(r) for r in mm.estimator['LTGL ($\ell_1$)'].latent_])
l6 = ([np.linalg.matrix_rank(r) for r in mm.estimator['LVGLASSO'].L])

In [ ]:
l1,l2,l3,l4,l5,l6 = utils.load_pickle(filename="ells.pkl")

In [ ]:
import collections
import matplotlib.pyplot as plt

f, (ax1, ax2) = plt.subplots(2,1, sharey=False, figsize=(10,5), dpi=600)

colors = ['white', 'lightblue', 'C7']
alpha = 0.95

counter=collections.Counter(l1)
ax1.bar(counter.keys(), np.array(counter.values())/len(l1), 
        alpha=alpha, width=0.24, label='LTGL ($\ell_2^2$)', color=colors[0], edgecolor='k')
counter=collections.Counter(l2)
ax1.bar(np.array(counter.keys())+0.25, np.array(counter.values())/len(l1), 
        alpha=alpha, width=0.24, label='LTGL ($\ell_1$)', color=colors[1], edgecolor='k')
counter=collections.Counter(l3)
ax1.bar(np.array(counter.keys())-0.25, np.array(counter.values())/len(l1), 
        alpha=alpha, width=0.24, label='LVGLASSO', color=colors[2], edgecolor='k')

ax1.set_xticks(range(0,30, 2))
#ax1.set_ylim(0,5)
ax1.axvline(20, c='r', ls='--')
ax1.set_xlabel(r'ranks of L obtained with ($p_2$)')
ax1.set_ylabel('frequency')
# ax1.set_xscale("log")
# ax1.set_xlim([10, 100])
ax1.xaxis.label.set_size(15)
ax1.yaxis.label.set_size(15)

#ax1.legend()
# ax0.legend(prop={'size': 10})
# ax0.set_title('bars with legend')


counter=collections.Counter(l4)
ax2.bar(counter.keys(), np.array(counter.values())/len(l4), 
        alpha=alpha, width=0.24, label='LTGL ($\ell_2^2$)', color=colors[0], edgecolor='k')
counter=collections.Counter(l5)
ax2.bar(np.array(counter.keys())+0.25,  
        np.array(counter.values())/len(l4), alpha=alpha, width=0.24, label='LTGL ($\ell_1$)', color=colors[1],
        edgecolor='k')
counter=collections.Counter(l6)
ax2.bar(np.array(counter.keys())-0.25,  
        np.array(counter.values())/len(l4), alpha=alpha, width=0.24, label='LVGLASSO', color=colors[2],
       edgecolor='k')

ax2.set_xticks(range(0,30,2))
# ax2.set_xlim(2.5,6.7)
ax2.set_xlabel(r'ranks of L obtained with ($p_1$)')
ax2.set_ylabel('frequency')
ax2.xaxis.label.set_size(15)
ax2.yaxis.label.set_size(15)
ax2.axvline(5, c='r', ls='--')
ax1.legend(loc='upper left', fontsize='x-large')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib
f.savefig("ranks_distribution_vertical.pdf", dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
import collections
import matplotlib.pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10,2.6), dpi=600)

colors = ['white', 'lightblue', 'C7']
alpha = 0.5

counter=collections.Counter(l1)
ax1.plot(range(len(l1)), l1, 
        alpha=alpha, label='LTGL ($\ell_2^2$)', color=colors[0])
counter=collections.Counter(l2)
ax1.plot(np.arange(len(l1))+.2, l2, 
        alpha=alpha,  label='LTGL ($\ell_1$)', color=colors[1])
counter=collections.Counter(l3)
ax1.plot(np.arange(len(l1))+.4, l3,
        alpha=alpha, label='LVGLASSO', color=colors[2])

# ax1.set_xticks(range(15,25, 1))
#ax1.set_ylim(0,5)
ax1.axhline(20, c='r', ls='--')
ax1.set_xlabel(r'ranks of L obtained with ($p_2$)')
ax1.set_ylabel('frequency')
ax1.xaxis.label.set_size(15)
ax1.yaxis.label.set_size(15)

#ax1.legend()
# ax0.legend(prop={'size': 10})
# ax0.set_title('bars with legend')


counter=collections.Counter(l4)
ax2.plot(range(len(l4)), l4, 
        alpha=alpha,label='LTGL ($\ell_2^2$)', color=colors[0])
counter=collections.Counter(l5)
ax2.plot(np.arange(len(l4))+.2, l5,
        alpha=alpha,  label='LTGL ($\ell_1$)', color=colors[1])
counter=collections.Counter(l6)
ax2.plot(np.arange(len(l4))+.4, l6, alpha=alpha,label='LVGLASSO', color=colors[2])

# ax2.set_xticks(range(10))
# ax2.set_xlim(2.5,6.7)
ax2.set_xlabel(r'ranks of L obtained with ($p_1$)')
ax2.xaxis.label.set_size(15)
ax2.axhline(5, c='r', ls='--')
ax1.legend(loc='best', fontsize='large')
plt.tight_layout()
plt.show()